In [0]:
%load_ext autoreload
%autoreload 2


In [0]:
# carregamento de bibliotecas
import sys
import os
from pyspark.sql import SparkSession
root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

if root not in sys.path:
    sys.path.append(root)


from src.features.utils import *

In [0]:
# construção da tabela histórica de preços do etanol hidratado industrial
PATH_ARQUIVO_HISTORICO = "/Volumes/api-forecasting/bronze/arquivos_brutos/cepea-consulta-20251119.csv"
TABELA_HISTORICA = "`api-forecasting`.bronze.precos_etanol_historico_raw"

df =  carregar_tabela_spark(format='csv', 
                            path=PATH_ARQUIVO_HISTORICO,
                            sep=';', infer_schema=True)
df.show()



In [0]:
# sanitizacao dos nomes das colunas para persistência como delta
df_limpo = df.withColumnRenamed("ï»¿Data", "Data") \
            .withColumnRenamed("Ã vista R$", "A_vista_R$") \
            .withColumnRenamed("Ã vista US$", "A_vista_US$")

In [0]:
# visualização da tabela sanitizada
df_limpo.show(5)

In [0]:
# persistência da tabela na camada bronze
salvar_tabela_delta_spark(df_limpo, 
                          mode='overwrite',
                          merge_schema=False,
                          partitionby=['Data'], 
                          path=TABELA_HISTORICA)

In [0]:
%sql
-- TABELA CATALOGADA NA CAMADA BRONZE
SELECT *
FROM `api-forecasting`.bronze.precos_etanol_historico_raw
LIMIT 10